In [1]:
!pip install d3rlpy==2.0.3
!pip install gym
# !pip install -U transformers==4.30.0
# !pip install scikit-learn==1.0.2
import os
import sys


from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel, TimeSeriesTransformerForPrediction
import torch
import gym
import numpy as np
import pickle
import random
from torch.utils.data import Dataset, DataLoader
# from google.colab import drive
import d3rlpy
import pandas as pd
# drive.mount('/content/drive')

with open('../data/rl_data.pkl', 'rb') as handle:
    rl_data = pickle.load(handle)

In [2]:
sys.path.append(os.path.join("/localhome/pagand/projects/model_optimze_vessel", 'Prepration'))

In [3]:
rl_data[1]["observations"][0]

array([ 1.00000000e+00,  1.73800000e+02,  5.78201792e-02, -3.14107591e-02,
        9.99506560e-01,  1.25905353e+00,  4.07498063e-01,  0.00000000e+00,
        0.00000000e+00,  9.31828385e-03,  0.00000000e+00,  2.13797399e-01,
        1.00000000e+00,  1.00000000e+00,  2.50000000e-01,  2.50000000e-01,
        3.74466392e-01,  1.86616923e-01,  4.21229992e-04,  2.32591947e-03])

In [4]:
import gym
from gym import spaces
import pickle
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import torch
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerForPrediction


In [5]:


observations = []
rewards_stage1 = []
rewards_stage2 = []
rewards_stage3 = []
actions = []
terminations = []
# for i in range(len(rl_data)):
for i in range(2):
    if i !=0:
        x = rl_data[i]
        obs = x["observations"]
        # new_obs = np.zeros([len(obs-25), 185])
        # for j in range(25, len(obs)):
            # new_obs[j-25] = np.concatenate([obs[j-25:j,[0,1,2,16,17,18,19]].reshape(1,-1), obs[j, 6:16].reshape(1,-1)], axis=1)
        observations.append(obs)
        rewards_stage1.append((np.sum(x["rewards"], axis=1)/4).reshape(-1,1))
        rewards_stage2.append((np.sum(x["rewards"][:, [0,1,3]], axis=1)/3).reshape(-1,1))
        rewards_stage3.append((np.sum(x["rewards"][:, [1,3]], axis=1)/2).reshape(-1,1))
        # rewards.append((np.sum(x["rewards"][:, [0,1,3]], axis=1)/3).reshape(-1,1))
        # rewards.append(x["rewards"][:,3].reshape(-1,1))
        terminations.append(x["termination"])
        x["actions"][:, 1] = x["actions"][:, 1]/360
        actions.append(x["actions"])


In [8]:
from VesselEnvironment import VesselEnvironment
csv_files = ["../data/H2N_top1.csv", "../data/N2H_top1.csv"]
stage1_env = VesselEnvironment(rl_data, "../data/fc_2_final.pt", "../data/longlat_3_final.pt", pickle.load(open('../data/minmax_scaler.pkl', 'rb')),csv_files ,reward_type="mimic")
stage2_env = VesselEnvironment(rl_data, "../data/fc_2_final.pt", "../data/longlat_3_final.pt", pickle.load(open('../data/minmax_scaler.pkl', 'rb')),csv_files, reward_type="top1")
stage3_env = VesselEnvironment(rl_data, "../data/fc_2_final.pt", "../data/longlat_3_final.pt", pickle.load(open('../data/minmax_scaler.pkl', 'rb')), csv_files, reward_type="conceptual")

In [9]:
import d3rlpy
dt = d3rlpy.algos.DecisionTransformerConfig(max_timestep=124, context_size=25, learning_rate=1e-6).create(device="cuda:0")

In [10]:
observations = np.vstack(observations)
rewards_stage1 = np.vstack(rewards_stage1)
rewards_stage2 = np.vstack(rewards_stage2)
rewards_stage3 = np.vstack(rewards_stage3)
actions = np.vstack(actions)
terminations = np.vstack(terminations)
dataset_stage1 = d3rlpy.dataset.MDPDataset(
    observations=observations,
    actions=actions,
    rewards=rewards_stage1,
    terminals=terminations,
)
dataset_stage2 = d3rlpy.dataset.MDPDataset(
    observations=observations,
    actions=actions,
    rewards=rewards_stage2,
    terminals=terminations,
)
dataset_stage3 = d3rlpy.dataset.MDPDataset(
    observations=observations,
    actions=actions,
    rewards=rewards_stage3,
    terminals=terminations,
)

In [11]:
# stage 1 training mimic
dt.fit(
   dataset_stage1,
   n_steps=42000,
   n_steps_per_epoch=6000,
   eval_env=stage2_env,
   eval_target_return=0,  # specify target environment return
)
dt.save_model('drive/MyDrive/dqn_stage1.pt')

2023-09-08 16:20:40 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(20,)]), action_signature=Signature(dtype=[dtype('float64')], shape=[(3,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=3)
2023-09-08 16:20:40 [info     ] Directory is created at d3rlpy_logs/DecisionTransformer_20230908162040
2023-09-08 16:20:40 [debug    ] Building models...
2023-09-08 16:20:40 [debug    ] Models have been built.
2023-09-08 16:20:40 [info     ] Parameters                     params={'observation_shape': [20], 'action_size': 3, 'config': {'type': 'decision_transformer', 'params': {'batch_size': 64, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'context_size': 25, 'learning_rate': 1e-06, 'encoder_factory': {'type': 'default

Epoch 1/7:   0%|          | 0/6000 [00:00<?, ?it/s]

2023-09-08 16:24:52 [info     ] DecisionTransformer_20230908162040: epoch=1 step=6000 epoch=1 metrics={'time_sample_batch': 0.0024981830517450967, 'time_algorithm_update': 0.022959060192108156, 'loss': 0.26705441424716264, 'time_step': 0.025536237001419066, 'environment': -203.59843510842117} step=6000
2023-09-08 16:24:52 [info     ] Model parameters are saved to d3rlpy_logs/DecisionTransformer_20230908162040/model_6000.d3


Epoch 2/7:   0%|          | 0/6000 [00:00<?, ?it/s]

2023-09-08 16:27:37 [info     ] DecisionTransformer_20230908162040: epoch=2 step=12000 epoch=2 metrics={'time_sample_batch': 0.0025697004397710163, 'time_algorithm_update': 0.019765571594238283, 'loss': 0.010562745163876874, 'time_step': 0.022402608195940654, 'environment': -151.87206979189574} step=12000
2023-09-08 16:27:37 [info     ] Model parameters are saved to d3rlpy_logs/DecisionTransformer_20230908162040/model_12000.d3


...

Epoch 6/7:   0%|          | 0/6000 [00:00<?, ?it/s]

2023-09-08 16:36:37 [info     ] DecisionTransformer_20230908162040: epoch=6 step=36000 epoch=6 metrics={'time_sample_batch': 0.002725160837173462, 'time_algorithm_update': 0.014930705547332763, 'loss': 0.00033320266874701095, 'time_step': 0.01772266924381256, 'environment': -182.05785818444048} step=36000
2023-09-08 16:36:37 [info     ] Model parameters are saved to d3rlpy_logs/DecisionTransformer_20230908162040/model_36000.d3


Epoch 7/7:   0%|          | 0/6000 [00:00<?, ?it/s]

2023-09-08 16:38:50 [info     ] DecisionTransformer_20230908162040: epoch=7 step=42000 epoch=7 metrics={'time_sample_batch': 0.002679446339607239, 'time_algorithm_update': 0.014611520210901896, 'loss': 0.0002395510635760729, 'time_step': 0.01735463547706604, 'environment': -137.13766941712618} step=42000
2023-09-08 16:38:50 [info     ] Model parameters are saved to d3rlpy_logs/DecisionTransformer_20230908162040/model_42000.d3


FileNotFoundError: [Errno 2] No such file or directory: 'drive/MyDrive/dqn_stage1.pt'

In [ ]:
# stage 2 training top 1
dt.fit(
   dataset_stage2,
   n_steps=42000,
   n_steps_per_epoch=6000,
   eval_env=stage2_env,
   eval_target_return=0,  # specify target environment return
)
dt.save_model('drive/MyDrive/dqn_stage2.pt')

In [ ]:
# stage 3 training conceptual rewards
dt.fit(
   dataset_stage3,
   n_steps=42000,
   n_steps_per_epoch=6000,
   eval_env=stage3_env,
   eval_target_return=0,  # specify target environment return
)
dt.save_model('drive/MyDrive/dqn_stage3.pt')

In [ ]:
actor = dt.as_stateful_wrapper(target_return=0)

# interaction
observation, reward = stage3_env.reset(), 0.0
while True:
    action = actor.predict(observation, reward)
    observation, reward, done, truncated, _ = stage3_env.step(action)
    if done or truncated:
        break